<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/A(10_1%EA%B0%95)_RNN_hihello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import numpy as np

# 데이터 준비
text = "hihello"
chars = sorted(set(text))  # 고유한 문자들 (['e', 'h', 'i', 'l', 'o'])
char_to_idx = {char: idx for idx, char in enumerate(chars)}  # 문자 -> 인덱스 변환
idx_to_char = {idx: char for idx, char in enumerate(chars)}  # 인덱스 -> 문자 변환

print("chars=", chars)
print("char_to_idx=", char_to_idx)
print("idx_to_char=", idx_to_char)

# 입력과 출력 데이터 생성
input_seq = [char_to_idx[char] for char in text[:-1]]  # 'hihell'에 해당하는 인덱스들
output_seq = [char_to_idx[char] for char in text[1:]]  # 'ihello'에 해당하는 인덱스들

# RNN 학습을 위해 입력 데이터를 (배치 크기, 타임스텝, 특징 수) 형태로 변환
input_seq = np.array(input_seq).reshape(1, -1)  # (1, 6)
output_seq = np.array(output_seq).reshape(1, -1)  # (1, 6)

# 하이퍼파라미터 정의
vocab_size = len(chars)  # 고유한 문자 개수
embedding_dim = 10  # 임베딩 차원
rnn_units = 50  # RNN 유닛 수

"""
1.
Embedding을 통해 단어 또는 문자 간의 의미적 유사성을 학습할 수 있습니다.
예를 들어:
만약 텍스트가 단어 기반이라면, "king"과 "queen"의 벡터 차이가 "man"과 "woman"의 벡터 차이와 유사하도록 학습될 수 있습니다.
문자 기반에서도 유사 문자는 유사 벡터로 학습될 가능성이 높습니다.

2.
.shape는 배열의 크기를 나타내는 속성입니다.
위 예제에서 input_seq.shape는 (2, 5)입니다.
첫 번째 차원(shape[0])은 샘플 수: 2개.
두 번째 차원(shape[1])은 시퀀스 길이: 5개.

3.
rnn_units는 RNN(Recurrent Neural Network) 레이어에서 사용하는 뉴런의 개수 또는 숨겨진 상태의 차원 크기를 나타냅니다.
이는 RNN 레이어가 입력 시퀀스 데이터를 처리하고 학습하는 데 사용하는 내부 상태의 크기를 결정합니다.
"""

"""
1. tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_seq.shape[1])
- vocab_size: 단어 집합(어휘)의 크기. 각 정수 토큰은 0부터 vocab_size-1까지의 값이어야 함.
- embedding_dim: 임베딩 벡터의 차원. 각 단어가 이 차원으로 매핑됨.
- input_length: 입력 시퀀스의 길이(타임스텝). input_seq.shape[1]을 사용해 동적으로 설정.
- 출력 형태: (배치 크기, 타임스텝, embedding_dim)

*** 임베딩의 역할 ***
텍스트 데이터에서 단어를 숫자로만 표현하면 각 숫자 간의 관계를 모델이 학습하기 어렵습니다.
예를 들어:
단어 "고양이" → 숫자 5
단어 "개" → 숫자 7
이렇게 정수로만 표현되면, 숫자 5와 7의 유사성을 모델이 이해할 수 없습니다.
Embedding 레이어는 각 단어를 의미적으로 유사한 고차원 벡터로 매핑해줍니다.

이를 통해:
"고양이"와 "개"는 유사한 의미를 가지므로 임베딩 벡터 공간에서 가까운 위치를 차지하게 됩니다.
반면, "사과"와 "고양이"는 멀리 떨어진 벡터로 표현됩니다.

2.  tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True)
- rnn_units: RNN의 유닛(노드) 수.
- return_sequences=True: 각 타임스텝의 출력을 반환하도록 설정. 이 옵션이 없으면 최종 출력만 반환됨.
- 출력 형태: (배치 크기, 타임스텝, rnn_units)
이 설정은 return_sequences=True 덕분에 모든 타임스텝에 대한 출력을 반환.

3. tf.keras.layers.Dense(vocab_size, activation='softmax')
- vocab_size: 출력 뉴런의 개수. 어휘 크기와 동일.
- activation='softmax': 각 뉴런의 값을 확률로 변환.
- 출력 형태: (배치 크기, 타임스텝, vocab_size)
"""
# 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_seq.shape[1]),
    tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# 모델 학습
model.fit(input_seq, output_seq, epochs=500)

# 학습된 모델로 다음 문자 예측
def predict_next_char(input_char):
    input_idx = np.array([char_to_idx[input_char]]).reshape(1, 1)
    pred = model.predict(input_idx)
    pred_char = idx_to_char[np.argmax(pred)]
    return pred_char

# 'h'로 시작하는 문자열에 대해 다음 문자 예측
start_char = 'h'
predicted_char = predict_next_char(start_char)
print(f"Next character after '{start_char}' is: '{predicted_char}'")

chars= ['e', 'h', 'i', 'l', 'o']
char_to_idx= {'e': 0, 'h': 1, 'i': 2, 'l': 3, 'o': 4}
idx_to_char= {0: 'e', 1: 'h', 2: 'i', 3: 'l', 4: 'o'}
Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.6131
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.5988
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 1.5847
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.5706
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 1.5562
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 1.5413
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.5258
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1.5094
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.4920
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 1.4735
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.4538
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.4327
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.4103
Epoch 14/5